目的：确定从ligand到target可能的signaling path
* 查找可能调控target gene的transcription factor，并且与配体下游最近（根据变得权重来定）
* 确定ligand到tf的最短路，最短路上的其他gene即为mediators
* 检查支持基因相互作用的数据来源

case study: infer signaling paths between CAF-ligand and p-EMT target

In [3]:
library(nichenetr)
library(tidyverse)


In [11]:
weighted_networks <- readRDS("./data/weighted_networks.rds")
ligand_tf_matrix <- readRDS("./data/ligand_tf_matrix.rds")

lr_network <- readRDS("./data/lr_network.rds")
sig_network <- readRDS("./data/signaling_network.rds")
gr_network <- readRDS("./data/gr_network.rds")

In [14]:
ligands_all <- "TGFB3" # 可以是一个list
targets_all <- c("TGFBI", "LAMC2", "TNC")

active_signaling_network = get_ligand_signaling_path(ligand_tf_matrix = ligand_tf_matrix, ligands_all = ligands_all, targets_all = targets_all, weighted_networks = weighted_networks)

# 为可视化，对权重进行normalization (0.75, 1.75)
active_signaling_network_min_max = active_signaling_network
active_signaling_network_min_max$sig = active_signaling_network_min_max$sig %>% mutate( weight=((weight-min(weight))/(max(weight)-min(weight))) + 0.75 )
active_signaling_network_min_max$gr  = active_signaling_network_min_max$gr %>% mutate( weight=((weight-min(weight))/(max(weight)-min(weight))) + 0.75 )

In [20]:
graph_min_max = diagrammer_format_signaling_graph(signaling_graph_list = active_signaling_network_min_max, ligands_all = ligands_all, targets_all = targets_all, sig_color = "indianred", gr_color = "steelblue")
#DiagrammeR::render_graph(graph_min_max, layout = "tree")

In [21]:
data_source_network = infer_supporting_datasources(signaling_graph_list = active_signaling_network,lr_network = lr_network, sig_network = sig_network, gr_network = gr_network)
head(data_source_network) 

from,to,source,database,layer
<chr>,<chr>,<chr>,<chr>,<chr>
SMAD1,TGFBI,regnetwork_source,regnetwork,regulatory
SMAD1,TGFBI,Remap_5,Remap,regulatory
SMAD2,LAMC2,harmonizome_CHEA,harmonizome_gr,regulatory
SMAD2,TGFBI,harmonizome_CHEA,harmonizome_gr,regulatory
SMAD2,TNC,harmonizome_CHEA,harmonizome_gr,regulatory
SMAD2,TNC,regnetwork_source,regnetwork,regulatory


Export to Cytoscape

In [23]:
potential_pathways <- bind_rows(active_signaling_network$sig %>% mutate(layer = "signaling"),
                                active_signaling_network$gr %>% mutate(layer = "regulatory"))

In [24]:
write.table(potential_pathways, "test.txt", sep="\t", quote=F)